# Gestión de tramas de moldeo
## Reconstrucción de las tramas de moldeo con las mezclas de Diam Process como soporte

In [1]:
import psycopg2
import pandas as pd
import os

<p>Creamos las variables que vamos a utilizar para seleccionar los valores entre fechas</p>
<p>El cóidigo funciona con una selección mínima del mes.</p>

In [2]:
fecha_inicio="2022/09/01"
fecha_fin="2022/09/30"

mes_referencia=fecha_inicio[5:7]
ano_referencia=fecha_inicio[2:4]


Función que permite sumar +1 a la cadena del mes y permite que siga siendo una cadena de dos caracteres Ejem. 01 -> 02

In [3]:
def suma_mes(mes):
    mes_cadena=str(int(mes)+1)
    if len(mes_cadena)==1:
        mes_cadena="0"+ mes_cadena
    return mes_cadena

Recuperamos todos los ficheros del periodo seleccionado +1 día ya que contiene mezclas del mes que estamso estudiando

In [4]:
tramas=[]
for (root,dirs,files) in os.walk('TRAMAS'):
    for name in files:
        if name[0:1]=="M":
            dia=name[12:14]
            mes=name[15:17]
            ano=name[20:22]
            if (mes_referencia==mes and ano_referencia == ano) or (dia=="01" and suma_mes(mes_referencia)==mes):
                tramas.append(root + "\\"+ os.path.join(name))

Concatenamos todos los ficheros en un dataframe

In [5]:
tramas_automatas = pd.concat((pd.read_csv(f, sep=";", encoding="latin-1", header=None, skiprows=1) for f in tramas), ignore_index=False)

1. Ponemos nombres a las columnas
2. Eliminamos aquellas columnas que no nos interesan
3. Ordenamos las columnas

In [6]:
tramas_automatas.columns =  ["Tipo", "Estacion", "Fecha", "Hora_inicio", "Mezcla", "Premezcla", "Batch", "Receta", "Kg_corcho", "Minutos_pesada", "Kg_DU", "Tiempo_vaciado", "Tiempo_inyeccion_agua", "Kg_Agua", "Tiempo_mezcla_agua", "Tiempo_inyeccion_cola", "Kg_cola", "Tiempo_mezcla_cola", "Tiempo_vaciado_mezcla", "Molde_inicial_ext1", "Molde_final_ext1", "Receta", "Molde_inicial_ext2", "Molde_final_ext2", "Tapones_buenos_g1", "Tapones_malos_g1", "Tiempo_extraccion_g1", "Tapones_buenos_g2", "Tapones_malos_g2", "Tiempo_extraccion_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2", "nada_1", "nada_2"]
tramas_automatas.drop(columns=["Tipo", "Hora_inicio", "Premezcla", "Batch", "Receta", "Minutos_pesada", "Tiempo_vaciado", "Tiempo_inyeccion_agua", "Kg_Agua", "Tiempo_mezcla_agua", "Tiempo_inyeccion_cola", "Tiempo_mezcla_cola", "Tiempo_vaciado_mezcla", "Molde_inicial_ext1", "Molde_final_ext1", "Receta", "Molde_inicial_ext2", "Molde_final_ext2", "Tiempo_extraccion_g1", "Tiempo_extraccion_g2","nada_1", "nada_2"], inplace=True)	
tramas_automatas=tramas_automatas.loc[:, ["Fecha", "Estacion", "Mezcla", "Kg_corcho", "Kg_cola", "Kg_DU", "Tapones_buenos_g1", "Tapones_buenos_g2", "Tapones_malos_g1", "Tapones_malos_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2"]]

Reseteamos el índice de las filas y eliminamos la columna indice que permanece al crear el nuevo.

In [7]:
tramas_automatas.reset_index(level=0, inplace=True)
tramas_automatas.drop(columns=["index"], inplace=True)

Creamos una seleccion booleana para filtrar las filas con errores en cantiddad de tapones, guardamos el porcentaje de líneas que vamos a reparar y las reparamos poniendo su valor a 0

In [8]:
tf=(tramas_automatas["Tapones_malos_g1"]==65492)
error_cantidad_malos= round((len(tf[tf==True])/len(tramas_automatas))*100,2)
tramas_automatas.loc[tf, "Tapones_malos_g1"]=0

Pasamos de unidades a millares, agrupamos los campos de tapones, eliminamos los que no necesitamos y ponemos el nombre de las modeadoras correctamente

In [9]:
tramas_automatas.loc[:, "Kg_corcho"]=tramas_automatas["Kg_corcho"]/1000
tramas_automatas.loc[:, "Kg_cola"]=tramas_automatas["Kg_cola"]/1000
tramas_automatas.loc[:, "Kg_DU"]=tramas_automatas["Kg_DU"]/1000
tramas_automatas.loc[:, "Tapones_buenos"]=(tramas_automatas["Tapones_buenos_g1"]+tramas_automatas["Tapones_buenos_g2"])/1000
tramas_automatas.loc[:, "Tapones_malos"]=(tramas_automatas["Tapones_malos_g1"]+tramas_automatas["Tapones_malos_g2"])/1000
tramas_automatas.loc[:, "Tapones_buenos_descartados"]=(tramas_automatas["Tapones_buenos_descartados_g1"]+tramas_automatas["Tapones_buenos_descartados_g2"])/1000
tramas_automatas.drop(columns=["Tapones_buenos_g1", "Tapones_buenos_g2", "Tapones_malos_g1", "Tapones_malos_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2"], inplace=True)
tramas_automatas["Estacion"].replace([10], "MA10", inplace=True)
tramas_automatas["Estacion"].replace([1], "MA11", inplace=True)
tramas_automatas["Estacion"].replace([12], "MA12", inplace=True)
tramas_automatas["Estacion"].replace([13], "MA13", inplace=True)
tramas_automatas["Estacion"].replace([14], "MA14", inplace=True)
tramas_automatas["Estacion"].replace([15], "MA15", inplace=True)

Guardamos el total de registros y eliminamos los duplicados. 

In [10]:
total_registros=len(tramas_automatas)
tramas_automatas.drop_duplicates(["Fecha", "Estacion", "Mezcla"], keep="first", inplace=True)

1. Convertimos la fecha de cadena a tipo datetime
2. Ordenamos los valores
3. Borramos las filas de fechas que no pertenecen al periodo seleccionado

In [11]:
tramas_automatas["Fecha"]=pd.to_datetime(tramas_automatas["Fecha"], format="%d/%m/%y")
tramas_automatas.sort_values(by=["Fecha", "Estacion", "Mezcla"], inplace=True)
tramas_automatas=tramas_automatas.loc[(tramas_automatas["Fecha"]>=fecha_inicio) & (tramas_automatas["Fecha"]<=fecha_fin)]

Guardamos el pordcentaje de filas eliminadas.

In [12]:
error_mezclas_duplicadas=round((1-(len(tramas_automatas)/total_registros))*100,2)
error_mezclas_duplicadas

28.16

In [13]:
#create csv file with commain float fields
tramas_automatas.to_csv("tramas_generado.csv", sep=";", encoding="latin-1", index=False,decimal=",")

## Extracción de las mezclas de la BBDDD diam Process

In [14]:
sentencia_sql = """
SELECT
	production_date,
	LEFT ( NAME, 4 ) moldeadora,
	mixture_number,
	cod,
	SUM ( grain_kilos ) AS grano,
	SUM ( agglomerate_kilos ) AS cola,
	SUM ( du_kilos ) AS du,
	SUM ( mv1_p ) / COUNT ( mixture_number ) AS peso 
FROM
	mixtures_mixture
	INNER JOIN machines_machine ON machines_machine.ID = mixtures_mixture.machine_id
	INNER JOIN specifications_specification ON specifications_specification.ID = mixtures_mixture.specification_id 
WHERE
	production_date >= '""" + fecha_inicio +"""' 
	AND production_date <= '""" + fecha_fin +"""'
	AND (
		machine_id = 1 
		OR machine_id = 2 
		OR machine_id = 3 
		OR machine_id = 4 
		OR machine_id = 5 
		OR machine_id = 6 
		OR machine_id = 7 
		OR machine_id = 8 
		OR machine_id = 9 
		OR machine_id = 10 
		OR machine_id = 11 
		OR machine_id = 12 
	) 
GROUP BY
	production_date,
	mixture_number,
	cod,
	moldeadora 
ORDER BY
	production_date,
	mixture_number"""

Ejecutamos la extracción de los datos.

In [15]:
conn = psycopg2.connect(host="10.0.1.36", database="diam", user="diam", password="diam")
cur = conn.cursor()
cur.execute( sentencia_sql )


Añadimos la información a un dataframe y configuramos las columnas

In [16]:
mezclas_DP= pd.DataFrame(cur.fetchall())
conn.close()
mezclas_DP.columns =  ["Fecha", "Moldeadora", "Mezcla", "Articulo", "kg_grano", "kg_cola", "Kg_du", "Peso"]

In [17]:
mezclas_DP

,Fecha,Moldeadora,Mezcla,Articulo,kg_grano,kg_cola,Kg_du,Peso
0,2022-09-01,MA13,1,BRUTCF038R027,65.0,18.2,3.90,7.270
1,2022-09-01,MA11,1,BRUTCF039R006,65.0,32.5,3.24,7.330
2,2022-09-01,MA12,1,BRUTCF041R028,65.0,20.8,4.22,8.070
3,2022-09-01,MA14,1,BRUTCF054R016,60.0,30.0,4.80,9.150
4,2022-09-01,MA13,2,BRUTCF038R027,65.0,18.2,3.90,7.300
...,...,...,...,...,...,...,...,...
8836,2022-09-26,MA12,9,BRUTCF041R028,65.0,20.8,4.22,8.125
8837,2022-09-26,MA11,9,BRUTCF109R006,50.0,25.0,2.76,7.225
8838,2022-09-26,MA13,9,BRUTCF109R027,50.0,25.0,2.76,7.430
8839,2022-09-26,MA12,10,BRUTCF041R028,65.0,20.8,4.22,8.100
